In [72]:
import pandas as pd
from ete3 import NCBITaxa

In [73]:
# 初始化 NCBITaxa
ncbi = NCBITaxa()

In [65]:
def get_taxonomy_info(taxid):
    """获取 taxonomy ID 对应的分类信息"""
    try:
        lineage = ncbi.get_lineage(taxid)  # 获取分类层级
        names = ncbi.get_taxid_translator(lineage)  # 获取分类名
        ranks = ncbi.get_rank(lineage)  # 获取分类级别

        # 生成分类信息字典
        rank_dict = {r: names[t] for t, r in ranks.items() if r in ["superkingdom", "phylum", "class", "order", "family", "genus", "species"]}
        
        # 按指定分类级别排列，确保返回 7 个分类
        taxonomy = [rank_dict.get(rank, "") for rank in ["superkingdom", "phylum", "class", "order", "family", "genus", "species"]]
        return taxonomy
    except Exception as e:
        print(f"Error fetching taxonomy for {taxid}: {e}")
        return [""] * 7  # 确保返回 7 个空值

def expand_species_accession(df):
    """拆分 SPECIES_ACCESSION 列，并在原始 df 基础上增加新行"""
    new_rows = []

    for _, row in df.iterrows():
        species_accession = row['SPECIES_ACCESSION']
        if pd.isna(species_accession):  # 处理 NaN
            new_rows.append(row.to_dict())
            continue

        taxids = [x.replace("NCBI:txid", "") for x in str(species_accession).split("{}") if x.startswith("NCBI:txid")]

        for taxid in taxids:
            new_row = row.to_dict()  # 复制原始行数据
            new_row['SPECIES_ACCESSION'] = f"NCBI:txid{taxid}"
            new_rows.append(new_row)

    return pd.DataFrame(new_rows)

def classify_species(df):
    """对扩展后的 df 进行分类"""
    taxonomy_data = []

    for _, row in df.iterrows():
        species_accession = str(row['SPECIES_ACCESSION'])  # 确保是字符串
        
        if pd.isna(row['SPECIES_ACCESSION']) or not species_accession.startswith("NCBI:txid"):
            taxonomy_info = [""] * 7  # 处理 NaN 和异常值
        else:
            taxid = species_accession.replace("NCBI:txid", "")
            taxonomy_info = get_taxonomy_info(int(taxid)) if taxid.isdigit() else [""] * 7

        new_row = row.to_dict()
        new_row.update(dict(zip(["superkingdom", "phylum", "class", "order", "family", "genus", "species"], taxonomy_info)))
        taxonomy_data.append(new_row)

    return pd.DataFrame(taxonomy_data)


In [57]:
# 读取 CSV
df = pd.read_csv("/Users/ejo/Desktop/chebi_compound.csv", encoding="latin1")

/var/folders/y6/5prks9dj0w11q5d9k3dq9qc40000gn/T/ipykernel_82336/1715102194.py:2: DtypeWarning: Columns (1,5,6,7,19,22,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/ejo/Desktop/chebi_compound.csv", encoding="latin1")


In [58]:
# 过滤掉 a, b, c 三列同时为空的行
df = df.dropna(subset=['FORMULA', 'MASS', 'MONOISOTOPIC_MASS', 'CHARGE', 'InChI', 'SPECIES', 'SPECIES_ACCESSION'], how='all')

In [59]:
df

,COMPOUND_ID,FORMULA,MASS,MONOISOTOPIC_MASS,CHARGE,InChI,SPECIES,SPECIES_ACCESSION,ID,STATUS,...,FOODB.ID,METACYC.ID,HMDB.ID,LIPIDMAPS.ID,KEGG_DRUG.ID,KEGG_GLYCAN.ID,WIKIPEDIA.ID,DRUGBANK.ID,PUBCHEM.ID,Synonyms
0,CHEBI:18357,C8H11NO3,169.17780,169.07389,0.0,InChI=1S/C8H11NO3/c9-4-8(12)5-1-2-6(10)7(11)3-...,Mus musculus,NCBI:txid10090,18357.0,C,...,NaN,NaN,HMDB0000216,NaN,D00076,NaN,Norepinephrine,DB00368,NaN,Noradrenaline{}L-Noradrenaline{}Norepinephrine...
1,CHEBI:3,(C4H6O2)n,NaN,NaN,NaN,NaN,Mus musculus,NCBI:txid10090,3.0,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,((R)-3-Hydroxybutanoyl)(n-2)
2,CHEBI:28234,C13H12O2,200.23320,200.08373,0.0,InChI=1S/C13H12O2/c14-11-6-5-9-7-8-3-1-2-4-10(...,NaN,NaN,28234.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(+)-(3S,4R)-cis-3,4-Dihydroxy-3,4-dihydrofluor..."
3,CHEBI:15399,C10H14O,150.21756,150.10447,0.0,InChI=1S/C10H14O/c1-7(2)9-5-4-8(3)10(11)6-9/h4...,NaN,NaN,15399.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(+)-(S)-Carvone{}Carvol{}(+)-Carvone{}Carvone{...
4,CHEBI:7,C10H16,136.23404,136.12520,0.0,"InChI=1S/C10H16/c1-7-4-5-8-9(6-7)10(8,2)3/h4,8...",NaN,NaN,7.0,C,...,NaN,CPD-8756,NaN,LMPR0102120021,NaN,NaN,NaN,NaN,NaN,"(+)-3-Carene{}(1S,6R)-3,7,7-trimethylbicyclo[4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148478,CHEBI:85274,NaN,NaN,NaN,NaN,NaN,Streptomyces pristinaespiralis,NCBI:txid38300,85274.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pristinamycine{}pristinamycin
148479,CHEBI:85331,NaN,NaN,NaN,NaN,NaN,Pantoea agglomerans,NCBI:txid549,85331.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dapdiamides
148480,CHEBI:87032,NaN,NaN,NaN,NaN,NaN,Chryseobacterium artocarpi{}Flexibacter elegans,NCBI:txid1414727{}NCBI:txid997,87032.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148481,CHEBI:8735,NaN,NaN,NaN,NaN,NaN,Mus musculus,NCBI:txid10090,8735.0,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R{}Methyl group acceptor


In [60]:
# 删除 a 列的重复值，仅保留第一次出现的行
df = df.drop_duplicates(subset='COMPOUND_ID', keep='first')

In [61]:
df

,COMPOUND_ID,FORMULA,MASS,MONOISOTOPIC_MASS,CHARGE,InChI,SPECIES,SPECIES_ACCESSION,ID,STATUS,...,FOODB.ID,METACYC.ID,HMDB.ID,LIPIDMAPS.ID,KEGG_DRUG.ID,KEGG_GLYCAN.ID,WIKIPEDIA.ID,DRUGBANK.ID,PUBCHEM.ID,Synonyms
0,CHEBI:18357,C8H11NO3,169.17780,169.07389,0.0,InChI=1S/C8H11NO3/c9-4-8(12)5-1-2-6(10)7(11)3-...,Mus musculus,NCBI:txid10090,18357.0,C,...,NaN,NaN,HMDB0000216,NaN,D00076,NaN,Norepinephrine,DB00368,NaN,Noradrenaline{}L-Noradrenaline{}Norepinephrine...
1,CHEBI:3,(C4H6O2)n,NaN,NaN,NaN,NaN,Mus musculus,NCBI:txid10090,3.0,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,((R)-3-Hydroxybutanoyl)(n-2)
2,CHEBI:28234,C13H12O2,200.23320,200.08373,0.0,InChI=1S/C13H12O2/c14-11-6-5-9-7-8-3-1-2-4-10(...,NaN,NaN,28234.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(+)-(3S,4R)-cis-3,4-Dihydroxy-3,4-dihydrofluor..."
3,CHEBI:15399,C10H14O,150.21756,150.10447,0.0,InChI=1S/C10H14O/c1-7(2)9-5-4-8(3)10(11)6-9/h4...,NaN,NaN,15399.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(+)-(S)-Carvone{}Carvol{}(+)-Carvone{}Carvone{...
4,CHEBI:7,C10H16,136.23404,136.12520,0.0,"InChI=1S/C10H16/c1-7-4-5-8-9(6-7)10(8,2)3/h4,8...",NaN,NaN,7.0,C,...,NaN,CPD-8756,NaN,LMPR0102120021,NaN,NaN,NaN,NaN,NaN,"(+)-3-Carene{}(1S,6R)-3,7,7-trimethylbicyclo[4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148478,CHEBI:85274,NaN,NaN,NaN,NaN,NaN,Streptomyces pristinaespiralis,NCBI:txid38300,85274.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pristinamycine{}pristinamycin
148479,CHEBI:85331,NaN,NaN,NaN,NaN,NaN,Pantoea agglomerans,NCBI:txid549,85331.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dapdiamides
148480,CHEBI:87032,NaN,NaN,NaN,NaN,NaN,Chryseobacterium artocarpi{}Flexibacter elegans,NCBI:txid1414727{}NCBI:txid997,87032.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148481,CHEBI:8735,NaN,NaN,NaN,NaN,NaN,Mus musculus,NCBI:txid10090,8735.0,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R{}Methyl group acceptor


In [62]:
# **第一步：扩展 `df`，在原始 `df` 上增加行**
df_expanded = expand_species_accession(df)

In [63]:
def classify_species(df):
    """对扩展后的 df 进行分类"""
    taxonomy_data = []

    for _, row in df.iterrows():
        species_accession = str(row['SPECIES_ACCESSION'])  # 确保是字符串
        
        if pd.isna(row['SPECIES_ACCESSION']) or not species_accession.startswith("NCBI:txid"):
            taxonomy_info = [""] * 7  # 处理 NaN 和异常值
        else:
            taxid = species_accession.replace("NCBI:txid", "")
            taxonomy_info = get_taxonomy_info(int(taxid)) if taxid.isdigit() else [""] * 7

        new_row = row.to_dict()
        new_row.update(dict(zip(["superkingdom", "phylum", "class", "order", "family", "genus", "species"], taxonomy_info)))
        taxonomy_data.append(new_row)

    return pd.DataFrame(taxonomy_data)


df_expanded

,COMPOUND_ID,FORMULA,MASS,MONOISOTOPIC_MASS,CHARGE,InChI,SPECIES,SPECIES_ACCESSION,ID,STATUS,...,FOODB.ID,METACYC.ID,HMDB.ID,LIPIDMAPS.ID,KEGG_DRUG.ID,KEGG_GLYCAN.ID,WIKIPEDIA.ID,DRUGBANK.ID,PUBCHEM.ID,Synonyms
0,CHEBI:18357,C8H11NO3,169.17780,169.07389,0.0,InChI=1S/C8H11NO3/c9-4-8(12)5-1-2-6(10)7(11)3-...,Mus musculus,NCBI:txid10090,18357.0,C,...,NaN,NaN,HMDB0000216,NaN,D00076,NaN,Norepinephrine,DB00368,NaN,Noradrenaline{}L-Noradrenaline{}Norepinephrine...
1,CHEBI:3,(C4H6O2)n,NaN,NaN,NaN,NaN,Mus musculus,NCBI:txid10090,3.0,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,((R)-3-Hydroxybutanoyl)(n-2)
2,CHEBI:28234,C13H12O2,200.23320,200.08373,0.0,InChI=1S/C13H12O2/c14-11-6-5-9-7-8-3-1-2-4-10(...,NaN,NaN,28234.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(+)-(3S,4R)-cis-3,4-Dihydroxy-3,4-dihydrofluor..."
3,CHEBI:15399,C10H14O,150.21756,150.10447,0.0,InChI=1S/C10H14O/c1-7(2)9-5-4-8(3)10(11)6-9/h4...,NaN,NaN,15399.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(+)-(S)-Carvone{}Carvol{}(+)-Carvone{}Carvone{...
4,CHEBI:7,C10H16,136.23404,136.12520,0.0,"InChI=1S/C10H16/c1-7-4-5-8-9(6-7)10(8,2)3/h4,8...",NaN,NaN,7.0,C,...,NaN,CPD-8756,NaN,LMPR0102120021,NaN,NaN,NaN,NaN,NaN,"(+)-3-Carene{}(1S,6R)-3,7,7-trimethylbicyclo[4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152963,CHEBI:87032,NaN,NaN,NaN,NaN,NaN,Chryseobacterium artocarpi{}Flexibacter elegans,NCBI:txid1414727,87032.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152964,CHEBI:87032,NaN,NaN,NaN,NaN,NaN,Chryseobacterium artocarpi{}Flexibacter elegans,NCBI:txid997,87032.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152965,CHEBI:8735,NaN,NaN,NaN,NaN,NaN,Mus musculus,NCBI:txid10090,8735.0,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R{}Methyl group acceptor
152966,CHEBI:87442,NaN,NaN,NaN,NaN,NaN,Synechocystis sp.{}Synechocystis sp.,NCBI:txid1148,87442.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
df_classified = classify_species(df_expanded)


/opt/anaconda3/lib/python3.11/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 155871 was translated into 1934393
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/opt/anaconda3/lib/python3.11/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 1000424 was translated into 52477
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/opt/anaconda3/lib/python3.11/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 74465 was translated into 2771340
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/opt/anaconda3/lib/python3.11/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 155873 was translated into 2587399
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/opt/anaconda3/lib/python3.11/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:243: UserWarning: taxid 28575 was translat

In [71]:
df_expanded

,COMPOUND_ID,FORMULA,MASS,MONOISOTOPIC_MASS,CHARGE,InChI,SPECIES,SPECIES_ACCESSION,ID,STATUS,...,FOODB.ID,METACYC.ID,HMDB.ID,LIPIDMAPS.ID,KEGG_DRUG.ID,KEGG_GLYCAN.ID,WIKIPEDIA.ID,DRUGBANK.ID,PUBCHEM.ID,Synonyms
0,CHEBI:18357,C8H11NO3,169.17780,169.07389,0.0,InChI=1S/C8H11NO3/c9-4-8(12)5-1-2-6(10)7(11)3-...,Mus musculus,NCBI:txid10090,18357.0,C,...,NaN,NaN,HMDB0000216,NaN,D00076,NaN,Norepinephrine,DB00368,NaN,Noradrenaline{}L-Noradrenaline{}Norepinephrine...
1,CHEBI:3,(C4H6O2)n,NaN,NaN,NaN,NaN,Mus musculus,NCBI:txid10090,3.0,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,((R)-3-Hydroxybutanoyl)(n-2)
2,CHEBI:28234,C13H12O2,200.23320,200.08373,0.0,InChI=1S/C13H12O2/c14-11-6-5-9-7-8-3-1-2-4-10(...,NaN,NaN,28234.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(+)-(3S,4R)-cis-3,4-Dihydroxy-3,4-dihydrofluor..."
3,CHEBI:15399,C10H14O,150.21756,150.10447,0.0,InChI=1S/C10H14O/c1-7(2)9-5-4-8(3)10(11)6-9/h4...,NaN,NaN,15399.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(+)-(S)-Carvone{}Carvol{}(+)-Carvone{}Carvone{...
4,CHEBI:7,C10H16,136.23404,136.12520,0.0,"InChI=1S/C10H16/c1-7-4-5-8-9(6-7)10(8,2)3/h4,8...",NaN,NaN,7.0,C,...,NaN,CPD-8756,NaN,LMPR0102120021,NaN,NaN,NaN,NaN,NaN,"(+)-3-Carene{}(1S,6R)-3,7,7-trimethylbicyclo[4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152963,CHEBI:87032,NaN,NaN,NaN,NaN,NaN,Chryseobacterium artocarpi{}Flexibacter elegans,NCBI:txid1414727,87032.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152964,CHEBI:87032,NaN,NaN,NaN,NaN,NaN,Chryseobacterium artocarpi{}Flexibacter elegans,NCBI:txid997,87032.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152965,CHEBI:8735,NaN,NaN,NaN,NaN,NaN,Mus musculus,NCBI:txid10090,8735.0,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R{}Methyl group acceptor
152966,CHEBI:87442,NaN,NaN,NaN,NaN,NaN,Synechocystis sp.{}Synechocystis sp.,NCBI:txid1148,87442.0,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
output_file = "/Users/ejo/Desktop/chebi_origin.csv"

In [70]:
# **保存为 CSV**
df_expanded.to_csv(output_file, index=False, encoding="utf-8")

In [75]:
df_classified.to_csv(output_file, index=False, encoding="utf-8")